# RAG

Build a simple Retrieval-Augmented Generation pipeline to demonstrate its working.

Steps:
1. Creating a knowledge base : Use the sentence BERT model to convert the document into a list of embeddings.
2. Creating an index : Using FAISS (Facebook AI similarity search) library, create an index that can be queried for its nearest neighbors
3. Retrieval: Given a query, encode it and return the most similar embeddings from the knowledge base
4. Generation: Use GPT-2 for generating a response

## Creating a Knowledge Base

To build out a simple RAG, use a few text documents as the knowledge base.
Run sentence BERT on these documents, on overlapping blocks of text and store the vectors.
Accept a query from the user
Run sentence BERT on the query, retrieve the "K" most relevant embeddings from the knowledge base using L2 distance (FAISS)
Provide the user query, retrieved embeddings to GPT-2 and generate a response.


For the knowledge base, I copied the text about Tour de France from Wikipedia and saved it in a text file.

In [1]:
from rag import RAG
import torch
from sentenceBERT import sentenceBERT
from bert_config import BERTConfig
from gpt import GPT
from gpt_config import GPTConfig

device = "cuda"
ckpt_path = "out/bert_ckpt_train.pt"
ckpt = torch.load(ckpt_path)
embedding_model_config = BERTConfig()
embedding_model = sentenceBERT(embedding_model_config)
embedding_model.to(device)

generate_model = GPT.from_pretrained(GPTConfig(block_size=1024,use_lora=False))
generate_model.to(device)
embedding_model_size = embedding_model_config.embedding_size
sentence_size = 3
overlap_size = 1
k = 5

rag = RAG(embedding_model,embedding_model_size,generate_model,sentence_size,overlap_size,k)
docs = ["tdf.txt"]
rag.add_to_knowledge_base(docs)


/tmp/ipykernel_1743987/3278462792.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path)


Loading pre-trained weights for bert
Loading pre-trained weights for gpt2
Number of parameters: 123.65M


In [3]:
query = "Tell me about the yellow jersey"
response = rag.get_response(query)
print(response)

Query: Tell me about the yellow jersey, Response:  This was the last rider to wear the yellow jersey. Tell me about the red lantern Greeting: After many years, the team organizer at this time, Jean-Pierre Rolland, introduced the yellow and red light as a symbol of solidarity.
